# Digits Generation with Auto Encoder
## Import Packages

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import time
import pandas as pd

## Utilities

In [ ]:
def sample_images(images, row_count, column_count):
    fig, axs = plt.subplots(row_count, column_count, figsize=(10,10))
    for i in range(row_count):
        for j in range(column_count):
            axs[i,j].imshow(images[i * column_count + j])
            axs[i,j].axis('off')
    plt.show()

## Import Datasets

In [ ]:
batch_size = 100
n_epochs = 10
image_width = 32

In [ ]:
def preproces_image(item):
    image = item["image"]
    image = tf.cast(image, "float")  / 255.0
    image =tf.image.resize(image, (image_width, image_width))
    return image, image

In [ ]:
train = tfds.load("mnist", split='train', as_supervised=False).map(preproces_image).shuffle(1024).batch(batch_size, drop_remainder=True).prefetch(1)
test = tfds.load("mnist", split='test', as_supervised=False).map(preproces_image).shuffle(1024).batch(batch_size, drop_remainder=True).prefetch(1)

In [ ]:
for images, _ in train.take(1):
    sample_images(images, 10, 10)

## Build the Auto Encoder

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
AE = keras.models.Sequential([
    keras.layers.Conv2D(32, 3, strides=2, padding="same", activation="relu", input_shape=(image_width, image_width, 1)),
    keras.layers.Conv2D(64, 3, strides=2, padding="same", activation="relu"),
    keras.layers.Conv2D(128, 3, strides=2, padding="same", activation="relu"),
    keras.layers.UpSampling2D(2),
    keras.layers.Conv2D(64, 3, padding="same", activation="relu"),
    keras.layers.UpSampling2D(2),
    keras.layers.Conv2D(32, 3, padding="same", activation="relu"),
    keras.layers.UpSampling2D(2),
    keras.layers.Conv2D(1, 3, padding="same", activation="relu")
])
AE.summary()

In [ ]:
AE.compile(loss="mean_squared_error", optimizer="adam", metrics=["mae"])

## Train the Model

In [ ]:
history = AE.fit(train, epochs=20)

## Model Evaluation

In [ ]:
pd.DataFrame(history.history).plot()

## 

In [ ]:
mse, mae = AE.evaluate(test)
print("MSE:", mse)
print("MAE:", mae)

## Encode the test images

In [ ]:
result = AE.predict(test)
result.shape

## 

In [ ]:
indices = np.random.choice(result.shape[0], 100)
sample_images(result[indices], 10, 10)

## Save the Model

In [ ]:
AE.save("AE.h5")